In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import sys

# reference main directory in existing folder
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import pandas as pd 
import sys
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from src.paths import RAW_DATA_DIR, CLEANED_DATA_DIR, QUADRANT_DATA_DIR
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# import z score library
from scipy.stats import zscore

# show all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# import data
#savant = pd.read_csv(RAW_DATA_DIR / 'savant_pitch_level.csv')
#data_dictionary = pd.read_excel(RAW_DATA_DIR / 'data_dictionary.xlsx')
#fangraph = pd.read_csv(RAW_DATA_DIR / 'fangraphs_season_level.csv')
fangraph = pd.read_csv(CLEANED_DATA_DIR / 'fangraphs_engineered_merged.csv')

# Import PCA quadrant data
q1_all_pitchers = pd.read_excel(QUADRANT_DATA_DIR / 'quadrant1_SP_RP_EQUAL.xlsx')
q2_all_pitchers = pd.read_excel(QUADRANT_DATA_DIR / 'quadrant2_SP_MAJORITY.xlsx')
q3_all_pitchers = pd.read_excel(QUADRANT_DATA_DIR / 'quadrant3_RP_MAJORITY.xlsx')
q4_all_pitchers = pd.read_excel(QUADRANT_DATA_DIR / 'quadrant4_RP_MAJORITY.xlsx')

# Rename columns to 'Stats'
q1_all_pitchers.rename(columns={'Unnamed: 0': 'Stats'}, inplace=True)
q2_all_pitchers.rename(columns={'Unnamed: 0': 'Stats'}, inplace=True)
q3_all_pitchers.rename(columns={'Unnamed: 0': 'Stats'}, inplace=True)
q4_all_pitchers.rename(columns={'Unnamed: 0': 'Stats'}, inplace=True)


## Utilizing Z-Scores for Pitcher Categorization
Z-Scores are a great way to normalize these metrics, allowing us to compare pitchers across different metrics effectively. By transforming data into a uniform scale, Z-scores enable a fair assessment of a pitcher's performance relative to the average, regardless of the specific metric. This normalization is crucial because it removes the inherent biases and variances that exist in raw statistical data. Additionally, Z-scores facilitate the identification of exceptional performances, both positive and negative, by highlighting how far an individual's performance deviates from the norm. Finally, this method simplifies the process of combining multiple metrics into a single performance score, ensuring that each metric contributes equally to the final evaluation without any one metric dominating due to scale differences.

In [3]:
# define starting pitchers from Throws column
starting_pitchers = fangraph[fangraph['Role'] == 'SP']
relief_pitchers = fangraph[fangraph['Role'] == 'RP']

In [4]:
starting_pitchers.shape

(1143, 326)

In [5]:
# Import feature loadings
q1 = pd.read_excel(QUADRANT_DATA_DIR / 'quadrant1_SP_RP_EQUAL.xlsx')
q2 = pd.read_excel(QUADRANT_DATA_DIR / 'quadrant2_SP_MAJORITY.xlsx')
q3 = pd.read_excel(QUADRANT_DATA_DIR / 'quadrant3_RP_MAJORITY.xlsx')
q4 = pd.read_excel(QUADRANT_DATA_DIR / 'quadrant4_RP_MAJORITY.xlsx')

# Rename columns to 'Stats'
q1.rename(columns={'Unnamed: 0': 'Stats'}, inplace=True)
q2.rename(columns={'Unnamed: 0': 'Stats'}, inplace=True)
q3.rename(columns={'Unnamed: 0': 'Stats'}, inplace=True)
q4.rename(columns={'Unnamed: 0': 'Stats'}, inplace=True)

In [6]:
import pandas as pd
from scipy.stats import zscore
from src.const import quadrant1_weights, quadrant2_weights, quadrant3_weights, quadrant4_weights

def categorize_pitchers(pitchers_df, pitcher_type, weights):
    """
    Categorizes pitchers into elite, strong, average, and suboptimal groups based on weighted z-scores of given metrics.
    
    Parameters:
    - pitchers_df (pd.DataFrame): DataFrame containing pitcher data.
    - pitcher_type (str): Type of pitcher ('SP' for starting pitchers or 'RP' for relief pitchers).
    - weights (dict): Dictionary of weights for different metrics.

    Returns:
    - Tuple of DataFrames: (elite, strong, average, suboptimal)
    """
    
    # Validate pitcher type
    pitcher_type = pitcher_type.upper()
    if pitcher_type not in ['SP', 'RP']:
        raise ValueError("pitcher_type must be 'SP' or 'RP'")

    # Check if the DataFrame is empty
    if pitchers_df.empty:
        raise ValueError("The provided DataFrame is empty.")
    
    # Check if the 'IP' column exists
    if 'IP' not in pitchers_df.columns:
        raise ValueError("'IP' column not found in DataFrame.")

    # Filter out pitchers below the mean IP
    mean_ip = pitchers_df['IP'].mean()
    pitchers_df = pitchers_df[pitchers_df['IP'] >= mean_ip]

    # Prepare for z-score calculation
    metric_cols = [col for col in weights.keys() if col in pitchers_df.columns]
    missing_cols = set(weights.keys()) - set(metric_cols)
    for col in missing_cols:
        print(f"Warning: Column {col} not found in DataFrame.")

    # Standardize the metric columns
    standard_scaler = StandardScaler()
    pitchers_df[metric_cols] = standard_scaler.fit_transform(pitchers_df[metric_cols])

    # Calculate z-scores for existing metrics
    z_scores = pitchers_df[metric_cols].apply(zscore)

    # Calculate weighted scores
    for col in metric_cols:
        z_scores[col] *= weights[col]
    pitchers_df['z_total_score'] = z_scores.sum(axis=1)

    # Define thresholds for categorization
    elite_threshold = pitchers_df['z_total_score'].quantile(.70)
    strong_threshold = pitchers_df['z_total_score'].quantile(.40)
    average_threshold = pitchers_df['z_total_score'].quantile(.10)

    # Categorize pitchers
    elite = pitchers_df[pitchers_df['z_total_score'] >= elite_threshold]
    strong = pitchers_df[(pitchers_df['z_total_score'] < elite_threshold) & (pitchers_df['z_total_score'] >= strong_threshold)]
    average = pitchers_df[(pitchers_df['z_total_score'] < strong_threshold) & (pitchers_df['z_total_score'] >= average_threshold)]
    suboptimal = pitchers_df[pitchers_df['z_total_score'] < average_threshold]

    return elite, strong, average, suboptimal

concatenated_weights = {**quadrant1_weights, **quadrant2_weights, **quadrant3_weights, **quadrant4_weights}


elite_sp, strong_sp, average_sp, suboptimal_sp = categorize_pitchers(starting_pitchers, 'SP', concatenated_weights)
elite_rp, strong_rp, average_rp, suboptimal_rp = categorize_pitchers(relief_pitchers, 'RP', concatenated_weights)



C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\3771004406.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitchers_df[metric_cols] = standard_scaler.fit_transform(pitchers_df[metric_cols])
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\3771004406.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pitchers_df['z_total_score'] = z_scores.sum(axis=1)
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\3771004406.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [7]:
"""# sort by year and total score
elite_sp = elite_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
strong_sp = strong_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
average_sp = average_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
suboptimal_sp = suboptimal_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])

elite_rp = elite_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
strong_rp = strong_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
average_rp = average_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
suboptimal_rp = suboptimal_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])

# Create a unique id for each pitcher
elite_sp['id'] = elite_sp['Name'] + elite_sp['Season'].astype(str)
strong_sp['id'] = strong_sp['Name'] + strong_sp['Season'].astype(str)
average_sp['id'] = average_sp['Name'] + average_sp['Season'].astype(str)
suboptimal_sp['id'] = suboptimal_sp['Name'] + suboptimal_sp['Season'].astype(str)

elite_rp['id'] = elite_rp['Name'] + elite_rp['Season'].astype(str)
strong_rp['id'] = strong_rp['Name'] + strong_rp['Season'].astype(str)
average_rp['id'] = average_rp['Name'] + average_rp['Season'].astype(str)
suboptimal_rp['id'] = suboptimal_rp['Name'] + suboptimal_rp['Season'].astype(str)
"""


"# sort by year and total score\nelite_sp = elite_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])\nstrong_sp = strong_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])\naverage_sp = average_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])\nsuboptimal_sp = suboptimal_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])\n\nelite_rp = elite_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])\nstrong_rp = strong_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])\naverage_rp = average_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])\nsuboptimal_rp = suboptimal_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])\n\n# Create a unique id for each pitcher\nelite_sp['id'] = elite_sp['Name'] + elite_sp['Season'].astype(str)\nstrong_sp['id'] = strong_sp['Name'] + strong_sp['Season'].astype(str)\naverage_sp['id'] = average_sp['Name'] + average_s

In [8]:
fangraph['id'] = fangraph['Name'] + fangraph['Season'].astype(str)

In [9]:
fangraph['Classification'] = 'unclassified'
fangraph['z_total_score'] = float('nan')

In [10]:
fangraph.head()

,PlayerId,pitcher,Name,NameASCII,Throws,Season,Age,Team,Role,G,GS,IP,TBF,W,L,CG,ShO,SV,BS,HLD,SD,MD,Pulls,ERA,R,ER,H,HR,SO,BB,IBB,HBP,WP,BK,Events,GB,LD,FB,IFFB,BU,IFH,BUH,Balls,Strikes,Pitches,RS,RS_per_9,K_pct,BB_pct,K_minus_BB_pct,K_per_9,BB_per_9,K_to_BB,H_per_9,HR_per_9,AVG,WHIP,BABIP,LOB_pct,GB_pct,LD_pct,FB_pct,IFFB_pct,GB_to_FB,HR_to_FB,IFH_pct,BUH_pct,FIP,xFIP,SIERA,tERA,kwERA,E_minus_F,RAR,WAR,Dollars,WPA,RE24,REW,pLI,inLI,gmLI,exLI,WPA_to_LI,Clutch,OSwing_pct,ZSwing_pct,Swing_pct,OContact_pct,ZContact_pct,Contact_pct,Zone_pct,FStrike_pct,SwStr_pct,CStr_pct,CSW_pct,ERA_minus,FIP_minus,xFIP_minus,Pace,RA9_WAR,BIP_Wins,LOB_Wins,FDP_Wins,Pull_pct,Cent_pct,Oppo_pct,Soft_pct,Med_pct,Hard_pct,TTO_pct,FRM,K_pct_plus,BB_pct_plus,K_per_9_plus,BB_per_9_plus,K_to_BB_plus,H_per_9_plus,HR_per_9_plus,AVG_plus,WHIP_plus,BABIP_plus,LOB_pct_plus,GB_pct_plus,LD_pct_plus,FB_pct_plus,HR_to_FB_pct_plus,Pull_pct_plus,Cent_pct_plus,Oppo_pct_plus,Med_pct_plus,Hard_pct_plus,EV,LA,Barrels,Barrel_pct,maxEV,HardHit,HardHit_pct,FA_pct,FAv,SL_pct,SLv,CT_pct,CTv,CB_pct,CBv,CH_pct,CHv,SF_pct,SFv,KN_pct,KNv,XX_pct,PO_pct,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB_per_c,wSL_per_c,wCT_per_c,wCB_per_c,wCH_per_c,wSF_per_c,wKN_per_c,FA_pct_sc,FC_pct_sc,FS_pct_sc,FO_pct_sc,SI_pct_sc,SL_pct_sc,CU_pct_sc,KC_pct_sc,EP_pct_sc,CH_pct_sc,SC_pct_sc,KN_pct_sc,UN_pct_sc,vFA_sc,vFC_sc,vFS_sc,vFO_sc,vSI_sc,vSL_sc,vCU_sc,vKC_sc,vEP_sc,vCH_sc,vSC_sc,vKN_sc,FA_X_sc,FC_X_sc,FS_X_sc,FO_X_sc,SI_X_sc,SL_X_sc,CU_X_sc,KC_X_sc,EP_X_sc,CH_X_sc,SC_X_sc,KN_X_sc,FA_Z_sc,FC_Z_sc,FS_Z_sc,FO_Z_sc,SI_Z_sc,SL_Z_sc,CU_Z_sc,KC_Z_sc,EP_Z_sc,CH_Z_sc,SC_Z_sc,KN_Z_sc,wFA_sc,wFC_sc,wFS_sc,wFO_sc,wSI_sc,wSL_sc,wCU_sc,wKC_sc,wEP_sc,wCH_sc,wSC_sc,wKN_sc,wFA_per_c_sc,wFC_per_c_sc,wFS_per_c_sc,wFO_per_c_sc,wSI_per_c_sc,wSL_per_c_sc,wCU_per_c_sc,wKC_per_c_sc,wEP_per_c_sc,wCH_per_c_sc,wSC_per_c_sc,wKN_per_c_sc,OSwing_pct_sc,ZSwing_pct_sc,Swing_pct_sc,OContact_pct_sc,ZContact_pct_sc,Contact_pct_sc,Zone_pct_sc,botERA,botOvr_CH,botStf_CH,botCmd_CH,botOvr_CU,botStf_CU,botCmd_CU,botOvr_FA,botStf_FA,botCmd_FA,botOvr_SI,botStf_SI,botCmd_SI,botOvr_SL,botStf_SL,botCmd_SL,botOvr_KC,botStf_KC,botCmd_KC,botOvr_FC,botStf_FC,botCmd_FC,botOvr_FS,botStf_FS,botCmd_FS,botOvr,botStf,botCmd,botxRV100,Stf_plus_CH,Loc_plus_CH,Pit_plus_CH,Stf_plus_CU,Loc_plus_CU,Pit_plus_CU,Stf_plus_FA,Loc_plus_FA,Pit_plus_FA,Stf_plus_SI,Loc_plus_SI,Pit_plus_SI,Stf_plus_SL,Loc_plus_SL,Pit_plus_SL,Stf_plus_KC,Loc_plus_KC,Pit_plus_KC,Stf_plus_FC,Loc_plus_FC,Pit_plus_FC,Stf_plus_FS,Loc_plus_FS,Pit_plus_FS,Stf_plus_FO,Loc_plus_FO,Pit_plus_FO,Stuff_plus,Location_plus,Pitching_plus,player_name,most_common_inning_SP,most_common_inning_RP,average_batters_faced_per_appearance_SP,average_innings_pitched_per_appearance_SP,average_batters_faced_per_appearance_RP,average_innings_pitched_per_appearance_RP,average_inning_SP,average_inning_RP,avg_inning_entry_SP,avg_inning_exit_SP,avg_inning_entry_RP,avg_inning_exit_RP,bad_walk_rate_SP,leadoff_walk_rate_SP,bad_walk_rate_RP,leadoff_walk_rate_RP,classified_role,id,Classification,z_total_score
0,18,491703,NeftalA­ Feliz,Neftali Feliz,R,2021,33,- - -,RP,5,0,4.0,19,0,1,0,0,0,1,0,0,2,4,9.000002,4,4,5,1,3,1,0,1,0,0,14,5,4,5,1,0,0,0,20,47,67,0,0.000000,0.1579,0.0526,0.1053,6.750002,2.250001,3.0000,11.250003,2.250001,0.294118,1.500000,0.307692,0.5357,0.357143,0.285714,0.357143,0.200000,1.000000,0.200000,0.000000,0.000000,6.420031,5.375481,4.519107,7.421088,4.889752,2.579971,-1.885527,-0.174990,-1.399920,-0.875182,-4.499,-0.420531,1.072752,0.609160,1.061280,0.263950,-0.137091,-0.678738,0.5000,0.7619,0.5821,0.7391,0.9375,0.8205,0.3134,0.6842,0.1045,0.1194,0.2239,213.324493,148.609662,126.817213,25.5870,-0.395077,-0.001708,-0.218379,-0.220087,0.4286,0.4286,0.1429,0.2143,0.5000,0.2857,0.2632,0.00000,67.733131,59.330599,75.367393,66.017851,76.108133,136.696882,183.262120,123.424821,115.998542,107.010757,74.114072,82.038249,137.175994,100.214061,145.942786,107.747591,121.632996,57.166409,97.430746,88.985942,84.847499,15.615007,3

In [11]:
import pandas as pd
from scipy.stats import zscore
from src.const import quadrant1_weights, quadrant2_weights, quadrant3_weights, quadrant4_weights

def categorize_pitchers(pitchers_df, pitcher_type, weights):
    """
    Categorizes pitchers into elite, strong, average, and suboptimal groups based on weighted z-scores of given metrics.
    
    Parameters:
    - pitchers_df (pd.DataFrame): DataFrame containing pitcher data.
    - pitcher_type (str): Type of pitcher ('SP' for starting pitchers or 'RP' for relief pitchers).
    - weights (dict): Dictionary of weights for different metrics.

    Returns:
    - Tuple of DataFrames: (elite, strong, average, suboptimal)
    """
    
    # Validate pitcher type
    pitcher_type = pitcher_type.upper()
    if pitcher_type not in ['SP', 'RP']:
        raise ValueError("pitcher_type must be 'SP' or 'RP'")

    # Check if the DataFrame is empty
    if pitchers_df.empty:
        raise ValueError("The provided DataFrame is empty.")
    
    # Check if the 'IP' column exists
    if 'IP' not in pitchers_df.columns:
        raise ValueError("'IP' column not found in DataFrame.")

    # Filter out pitchers below the mean IP
    #mean_ip = pitchers_df['IP'].mean()
    #pitchers_df = pitchers_df[pitchers_df['IP'] >= mean_ip]

    # Prepare for z-score calculation
    metric_cols = [col for col in weights.keys() if col in pitchers_df.columns]
    missing_cols = set(weights.keys()) - set(metric_cols)
    for col in missing_cols:
        print(f"Warning: Column {col} not found in DataFrame.")

    # Standardize the metric columns
    standard_scaler = StandardScaler()
    pitchers_df[metric_cols] = standard_scaler.fit_transform(pitchers_df[metric_cols])

    # Calculate z-scores for existing metrics
    z_scores = pitchers_df[metric_cols].apply(zscore)

    # Calculate weighted scores
    for col in metric_cols:
        z_scores[col] *= weights[col]
    pitchers_df['z_total_score'] = z_scores.sum(axis=1)

    # Define thresholds for categorization
    elite_threshold = pitchers_df['z_total_score'].quantile(.70)
    strong_threshold = pitchers_df['z_total_score'].quantile(.40)
    average_threshold = pitchers_df['z_total_score'].quantile(.10)

    # Categorize pitchers
    elite = pitchers_df[pitchers_df['z_total_score'] >= elite_threshold]
    strong = pitchers_df[(pitchers_df['z_total_score'] < elite_threshold) & (pitchers_df['z_total_score'] >= strong_threshold)]
    average = pitchers_df[(pitchers_df['z_total_score'] < strong_threshold) & (pitchers_df['z_total_score'] >= average_threshold)]
    suboptimal = pitchers_df[pitchers_df['z_total_score'] < average_threshold]

    return elite, strong, average, suboptimal

concatenated_weights = {**quadrant1_weights, **quadrant2_weights, **quadrant3_weights, **quadrant4_weights}


elite_sp, strong_sp, average_sp, suboptimal_sp = categorize_pitchers(starting_pitchers, 'SP', concatenated_weights)
elite_rp, strong_rp, average_rp, suboptimal_rp = categorize_pitchers(relief_pitchers, 'RP', concatenated_weights)



C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\206099726.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitchers_df[metric_cols] = standard_scaler.fit_transform(pitchers_df[metric_cols])


C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\206099726.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pitchers_df['z_total_score'] = z_scores.sum(axis=1)
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\206099726.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitchers_df['z_total_score'] = z_scores.sum(axis=1)
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\206099726.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [12]:
# sort by year and total score
elite_sp = elite_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
strong_sp = strong_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
average_sp = average_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
suboptimal_sp = suboptimal_sp.sort_values(['Season', 'z_total_score'], ascending=[False, False])

elite_rp = elite_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
strong_rp = strong_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
average_rp = average_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])
suboptimal_rp = suboptimal_rp.sort_values(['Season', 'z_total_score'], ascending=[False, False])

# Create a unique id for each pitcher
elite_sp['id'] = elite_sp['Name'] + elite_sp['Season'].astype(str)
strong_sp['id'] = strong_sp['Name'] + strong_sp['Season'].astype(str)
average_sp['id'] = average_sp['Name'] + average_sp['Season'].astype(str)
suboptimal_sp['id'] = suboptimal_sp['Name'] + suboptimal_sp['Season'].astype(str)

elite_rp['id'] = elite_rp['Name'] + elite_rp['Season'].astype(str)
strong_rp['id'] = strong_rp['Name'] + strong_rp['Season'].astype(str)
average_rp['id'] = average_rp['Name'] + average_rp['Season'].astype(str)
suboptimal_rp['id'] = suboptimal_rp['Name'] + suboptimal_rp['Season'].astype(str)


C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\3191421495.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  elite_sp['id'] = elite_sp['Name'] + elite_sp['Season'].astype(str)
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\3191421495.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  strong_sp['id'] = strong_sp['Name'] + strong_sp['Season'].astype(str)
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14772\3191421495.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

In [13]:
# Helper function to update fangraph DataFrame
def update_fangraph_classification(df, classification):
    # Create a temporary dictionary from the categorized DataFrame
    temp_dict = df.set_index('id')['z_total_score'].to_dict()
    
    # Update fangraph based on the unique id
    for pid, score in temp_dict.items():
        if pid in fangraph['id'].values:
            fangraph.loc[fangraph['id'] == pid, 'Classification'] = classification
            fangraph.loc[fangraph['id'] == pid, 'z_total_score'] = score

# Step 4: Update the fangraph DataFrame for each group
update_fangraph_classification(elite_sp, 'elite_sp')
update_fangraph_classification(strong_sp, 'strong_sp')
update_fangraph_classification(average_sp, 'average_sp')
update_fangraph_classification(suboptimal_sp, 'suboptimal_sp')

update_fangraph_classification(elite_rp, 'elite_rp')
update_fangraph_classification(strong_rp, 'strong_rp')
update_fangraph_classification(average_rp, 'average_rp')
update_fangraph_classification(suboptimal_rp, 'suboptimal_rp')




In [14]:
fangraph.isnull().sum()

PlayerId                                        0
pitcher                                         0
Name                                            0
NameASCII                                       0
Throws                                          0
Season                                          0
Age                                             0
Team                                            0
Role                                            0
G                                               0
GS                                              0
IP                                              0
TBF                                             0
W                                               0
L                                               0
CG                                              0
ShO                                             0
SV                                              0
BS                                              0
HLD                                             0


In [15]:
# save the fangraph DataFrame
fangraph.to_csv(CLEANED_DATA_DIR / 'fangraphs_zscores_merged.csv', index=False)